In [ ]:
!pip install catboost

In [2]:
import pandas as pd
import random
import os
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')

In [4]:
train = train.drop(['PRODUCT_ID', 'Y_Quality'], axis = 1)
test = test.drop(['PRODUCT_ID'], axis = 1)

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train['LINE'])
train['LINE'] = le.transform(train['LINE'])
test['LINE'] = le.transform(test['LINE'])

le.fit(train['PRODUCT_CODE'])
train['PRODUCT_CODE'] = le.transform(train['PRODUCT_CODE'])
test['PRODUCT_CODE'] = le.transform(test['PRODUCT_CODE'])

In [24]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

result = []
    
train_x = train.drop(["Y_Class"], axis = 1)
train_y = train["Y_Class"]

n = 0

for train_index, valid_index in skf.split(train_x, train_y):

    train_X = np.array(train_x)
    train_Y = np.array(train_y)

    X_train, X_valid = train_X[train_index], train_X[valid_index]
    y_train, y_valid = train_Y[train_index], train_Y[valid_index]

    model = CatBoostClassifier(iterations=1000,
                               random_seed=42,
                               learning_rate=0.1,
                               max_depth=5,
                               grow_policy="Depthwise",
                               verbose=0,
                               task_type="GPU")

    summary = model.select_features(
        X = X_train,
        y = y_train,
        features_for_select = '0-2876',
        eval_set=(X_valid, y_valid),
        num_features_to_select=300,
        steps=50,
        train_final_model=False,
        logging_level='Silent',
        plot=False
    )

    result.append(summary['selected_features'])

    print("result", n, "appended")
    n += 1

result 0 appended
result 1 appended
result 2 appended
result 3 appended
result 4 appended


In [25]:
result = np.array(result)
np.save('numpy_selected_feature_5fold_original_300', result)